1. Introduction
===================

In this project, we'll create [choropleth maps](https://en.wikipedia.org/wiki/Choropleth_map) to represent population data on brazilian municipalities of Northeast Region.

A choropleth map is a thematic map in which areas are shaded or patterned in proportion to the measurement of the statistical variable being displayed on the map, such as population density or per-capita income.

The [dataset](data/population_2017.csv) with population data comes from [IBGE - Instituto Brasileiro de Geografia e Estatística](https://downloads.ibge.gov.br/downloads_estatisticas.htm). We'll use python language with pandas library for data manipulation and folium library for creating maps with [Leaflet](http://leafletjs.com/).


In [137]:
import os
import folium
import json
import pandas as pd
from branca.colormap import linear
import numpy as np
from shapely.geometry import Polygon
from shapely.geometry import Point
from numpy import random

In [138]:
# dataset name
dataset_pop_2017 = os.path.join('data', 'population_2017.csv')

# read the data to a dataframe
data2017 = pd.read_csv(dataset_pop_2017)

# eliminate spaces in name of columns
data2017.columns = [cols.replace(' ', '_') for cols in data2017.columns]

# data for each state of Northeast Region
dataRN = data2017[data2017['UF'] == 'RN']
dataCE = data2017[data2017['UF'] == 'CE']
dataAL = data2017[data2017['UF'] == 'AL']
dataSE = data2017[data2017['UF'] == 'SE']
dataPI = data2017[data2017['UF'] == 'PI']
dataPB = data2017[data2017['UF'] == 'PB']
dataPE = data2017[data2017['UF'] == 'PE']
dataBA = data2017[data2017['UF'] == 'BA']
dataMA = data2017[data2017['UF'] == 'MA']

# data for Northeast Region
dataNE = data2017[data2017['UF'].isin(['RN','CE','AL','SE','PI','PB','PE','BA','MA'])]
dataNE.sort_values('NOME_DO_MUNICÍPIO').head()

,UF,COD._UF,COD._MUNIC,NOME_DO_MUNICÍPIO,POPULAÇÃO_ESTIMADA
891,CE,23.0,101.0,Abaiara,11605.0
1828,BA,29.0,207.0,Abaré,20189.0
1827,BA,29.0,108.0,Abaíra,9199.0
1465,PE,26.0,54.0,Abreu e Lima,99364.0
1829,BA,29.0,306.0,Acajutiba,15727.0


2. Geodata BR - Brazil
===================

[This project](https://github.com/tbrugz/geodata-br) contains files [Geojson](http://geojson.org/) with the perimeters
of brazilian municipalities grouped by state.

### States of Northeast Region
* AL / Alagoas - [geojson/geojs-27-mun.json](geojson/geojs-27-mun.json)
* BA / Bahia - [geojson/geojs-29-mun.json](geojson/geojs-29-mun.json)
* CE / Ceará - [geojson/geojs-23-mun.json](geojson/geojs-23-mun.json)
* MA / Maranhão - [geojson/geojs-21-mun.json](geojson/geojs-21-mun.json)
* PB / Paraíba - [geojson/geojs-25-mun.json](geojson/geojs-25-mun.json)
* PE / Pernambuco - [geojson/geojs-26-mun.json](geojson/geojs-26-mun.json)
* PI / Piauí - [geojson/geojs-22-mun.json](geojson/geojs-22-mun.json)
* RN / Rio Grande do Norte - [geojson/geojs-24-mun.json](geojson/geojs-24-mun.json)
* SE / Sergipe - [geojson/geojs-28-mun.json](geojson/geojs-28-mun.json)


3. Importing GeoJson files
===================

In [139]:
# searching the files in geojson/geojs-xx-mun.json
# load the data and use 'latin-1'encoding because the accent
geo_json_data_21 = json.load(open(os.path.join('geojson', 'geojs-21-mun.json'),encoding='latin-1'))
geo_json_data_22 = json.load(open(os.path.join('geojson', 'geojs-22-mun.json'),encoding='latin-1'))
geo_json_data_23 = json.load(open(os.path.join('geojson', 'geojs-23-mun.json'),encoding='latin-1'))
geo_json_data_24 = json.load(open(os.path.join('geojson', 'geojs-24-mun.json'),encoding='latin-1'))
geo_json_data_25 = json.load(open(os.path.join('geojson', 'geojs-25-mun.json'),encoding='latin-1'))
geo_json_data_26 = json.load(open(os.path.join('geojson', 'geojs-26-mun.json'),encoding='latin-1'))
geo_json_data_27 = json.load(open(os.path.join('geojson', 'geojs-27-mun.json'),encoding='latin-1'))
geo_json_data_28 = json.load(open(os.path.join('geojson', 'geojs-28-mun.json'),encoding='latin-1'))
geo_json_data_29 = json.load(open(os.path.join('geojson', 'geojs-29-mun.json'),encoding='latin-1'))

# http://cidades.ibge.gov.br/painel/historico.php?codmun=241030
# Presidente Juscelino city changes your name to Serra Caiada
geo_json_data_24['features'][112]['properties']['description'] = 'Serra Caiada'
geo_json_data_24['features'][112]['properties']['name'] = 'Serra Caiada'

In [140]:
# join geoJson files
geo_json_data = { "type": "FeatureCollection", "features": [ ] }
geo_json_data['features'].extend(geo_json_data_21['features'])
geo_json_data['features'].extend(geo_json_data_22['features'])
geo_json_data['features'].extend(geo_json_data_23['features'])
geo_json_data['features'].extend(geo_json_data_24['features'])
geo_json_data['features'].extend(geo_json_data_25['features'])
geo_json_data['features'].extend(geo_json_data_26['features'])
geo_json_data['features'].extend(geo_json_data_27['features'])
geo_json_data['features'].extend(geo_json_data_28['features'])
geo_json_data['features'].extend(geo_json_data_29['features'])

len(geo_json_data['features'])

1793

4. Drawing the Choropleth Map
======================

Now we need to convert the table into a dictionary in order to map a feature to it's population estimation.

In [141]:
population_data = dataNE.copy()
population_data['ID'] = dataNE['COD._UF'].astype('int').astype('str') + dataNE['COD._MUNIC'].astype('int').astype('str').apply(lambda x: x.zfill(5))

population_dict = population_data.set_index('ID')['POPULAÇÃO_ESTIMADA']
population_dict.head()

ID
2100055    111339.0
2100105      6486.0
2100154     12457.0
2100204     21673.0
2100303     26387.0
Name: POPULAÇÃO_ESTIMADA, dtype: float64

We need also to create a function that maps one value to a RGB color (of the form #RRGGBB). For this, we'll use colormap tools from branca.colormap.

In [142]:
colormap = linear.YlGnBu.scale(
    population_data.POPULAÇÃO_ESTIMADA.min(),
    population_data.POPULAÇÃO_ESTIMADA.max())

colormap

Now we can do the choropleth.

In [143]:
# Create a map object
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=7,
    tiles='Stamen Terrain'
)

# Configure geojson layer
folium.GeoJson(
    geo_json_data_21,
    name='Maranhão',
    style_function=lambda feature: {
        'fillColor': colormap(population_dict[feature['properties']['id']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(m)

folium.GeoJson(
    geo_json_data_22,
    name='Piauí',
    style_function=lambda feature: {
        'fillColor': colormap(population_dict[feature['properties']['id']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(m)

folium.GeoJson(
    geo_json_data_23,
    name='Ceará',
    style_function=lambda feature: {
        'fillColor': colormap(population_dict[feature['properties']['id']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(m)

folium.GeoJson(
    geo_json_data_24,
    name='Rio Grande do Norte',
    style_function=lambda feature: {
        'fillColor': colormap(population_dict[feature['properties']['id']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(m)

folium.GeoJson(
    geo_json_data_25,
    name='Paraíba',
    style_function=lambda feature: {
        'fillColor': colormap(population_dict[feature['properties']['id']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(m)

folium.GeoJson(
    geo_json_data_26,
    name='Pernambuco',
    style_function=lambda feature: {
        'fillColor': colormap(population_dict[feature['properties']['id']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(m)

folium.GeoJson(
    geo_json_data_27,
    name='Alagoas',
    style_function=lambda feature: {
        'fillColor': colormap(population_dict[feature['properties']['id']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(m)

folium.GeoJson(
    geo_json_data_28,
    name='Sergipe',
    style_function=lambda feature: {
        'fillColor': colormap(population_dict[feature['properties']['id']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(m)

folium.GeoJson(
    geo_json_data_29,
    name='Bahia',
    style_function=lambda feature: {
        'fillColor': colormap(population_dict[feature['properties']['id']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(m)

colormap.caption = 'Population estimation (2017)'
colormap.add_to(m)

folium.LayerControl().add_to(m)
m.save('map_geojson.html')

Then, in playing with keyword arguments, you can get a choropleth in (almost) one line :


In [144]:
# Create a map object
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=7,
    tiles='Stamen Terrain'
)

m.add_child(folium.LatLngPopup())

# create a threshold of legend
threshold_scale = np.linspace(population_data['POPULAÇÃO_ESTIMADA'].min(),
                              population_data['POPULAÇÃO_ESTIMADA'].max(), 6, dtype=int).tolist()

m.choropleth(
    geo_data=geo_json_data,
    data=population_data,
    columns=['ID', 'POPULAÇÃO_ESTIMADA'],
    key_on='feature.properties.id',
    fill_color='YlGn',
    legend_name='Population estimation (2017)',
    highlight=True,
    threshold_scale = threshold_scale
)

m.save('map_choropleth.html')